## Load Modules

In [0]:
# Install the PyDrive wrapper & import libraries.

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1GhyH4k9C4uPRnMAMKhJYOqa-V9Tqt4q8' ### File ID ###
data = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

## Read the data

In [2]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO(data.GetContentString())) 
data.head()

,Unnamed: 0,label,tweet
0,0,0.0,user when a father is dysfunctional and is s...
1,1,0.0,user user thanks for lyft credit i can t us...
2,2,0.0,bihday your majesty
3,3,0.0,model i love u take with u all the time in ...
4,4,0.0,factsguide society now motivation


## Load Flair and PyTorch Library

In [3]:
import torch
# !pip install flair
import flair
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-fi22hesk
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-fi22hesk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=18eb4d9afe3ebae6021176f616bc190e2ed7229251cc7b363ca86fd1f0b11bd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-23h9uvv7/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Found existing installation: flair 0.4.5
    Uninstalling flair-0.4.5:
      Successfully uninstalled flair-0.4.5


In [4]:
from flair.data import Sentence
# Create a sentence
sentence = Sentence('Blogs of Analytics Vidhya are Awesome.')
# Print the sentence to see what’s in it
print(sentence) # A Sentence is essentially a list of tokens

Sentence: "Blogs of Analytics Vidhya are Awesome."   [− Tokens: 6]


In [5]:
# Extract only the tweet column from the dataframe
text = data['tweet'] 
# Create a list fo the tweets called txt
txt = text.tolist()
print(txt[:10])

['  user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction     run', ' user  user thanks for  lyft credit i can t use cause they don t offer wheelchair vans in pdx      disapointed  getthanked', '  bihday your majesty', ' model   i love u take with u all the time in ur                                      ', ' factsguide  society now     motivation', '      huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo  ', '  user camping tomorrow  user  user  user  user  user  user  user danny   ', 'the next school year is the year for exams      can t think about that       school  exams    hate  imagine  actorslife  revolutionschool  girl', 'we won    love the land     allin  cavs  champions  cleveland  clevelandcavaliers      ', '  user  user welcome here    i m   it s so  gr    ']


## Word Embedding Using Flair

In [0]:
# Import the embeddings
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

# Initialise embeddings (un-comment to use others)
#glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
#bert_embedding = BertEmbedding()
#elmo_embedding = ElmoEmbedding()

# Stack the embeddings
stacked_embeddings = StackedEmbeddings( embeddings = [ 
                                                       flair_forward, 
                                                       flair_backward
                                                      ])